<ul>
    <li> <h2><b>Universidade de Brasília - UnB</b></h2> </li>
    <li> <b>Campus UnB Gama</b> </li>
    <li> <b>Disciplina: Processamento Digital de Sinais Financeiros</b> </li>
    <a href="https://www.linkedin.com/in/marcelino-andrade-b164b369/">
    <li> <b>Prof. Marcelino Monteiro de Andrade Dr.</b> </li>
</ul>


    Primeira (1/5) Atividade Prática Avaliativa
    Entrega da Atividade 01/10/2020 
    "A não entrega na data esperada reduz em 50% na nota da avaliação"


<div id="Tópicos">
    <h2>Questões</h2>
</div>


<div class="alert alert-block alert-info" style="margin-top: 20px">
<ol> 
<li> Colete os preços e volumes de um conjunto de ativos durante o período de 3 (três) anos, pertencentes a no mínimo 5 (cinco) Índices de Segmentos e Setoriais diferentes, e separe os 50 (cinquenta) ativos mais líquidos; </li><p></p>
    <li>Construa uma amostra de treinamento e outra de teste, sendo a primeira com os 2 (dois) primeiros anos e a segunda com o último anos; </li> <p></p>
    <li> Na amostra de treinamento, aplique uma clusterização não supervisionada (K-Means ou DBSCAN) e separe um ativo para cada clusters, com o critério que desejar (sugestão maior índice Sharpe); </li><p></p>
    <li> Aplique a Teoria do Portfolio Moderno de Markowitz nos ativos escolhidos no passo anterior, no sentido de alocar um capital de R$100.000,00; </li><p></p>
    <li> Verifique o resultado da alocação de capital com Markowitz, nos ativos separados na fase de treinamento, amostra de teste que representa o último ano da aquisição; </li><p></p>
    <li> Descreva suas impressores relativas ao processo e resultados encontrados. </li>
</ol>
</div>


### Questão 1: 

# Extração

Os dados das ações que compõem cada índice são extraídas diretamente da B3 através do seguinte [link](http://www.b3.com.br/pt_br/market-data-e-indices/indices/acoes-por-indice/), no fim da paǵina existe uma opção para realizar o download da planilha contendo as informações apresentadas.

In [84]:
import pandas as pd
import numpy as np

In [3]:
stock_by_indx = pd.read_html('data/AcoesIndices.SetDez.2020.xls')[0]

In [4]:
stock_by_indx.head()

RELA��O DE A��ES POR �NDICE - SETEMBRO A DEZEMBRO DE 2020Atualizado em 22/09/2020  \
                                                                            �ndices   
                                                                 Unnamed: 0_level_2   
0                                                 3M                                  
1                                         ABC BRASIL                                  
2                                       ADVANCED MIC                                  
3                                        ADVANCED-DH                                  
4                                        AES TIETE E                                  

                                                                       ...  \
                                                                       ...   
  Unnamed: 1_level_2    BDRX IBOV    IBRA IBXL IBXX ICO2 ICON    IDIV  ...   
0                DRN  MMMC34  NaN     NaN  NaN  NaN  NaN  NaN     NaN  ...   
1              PN N2     NaN  NaN   ABCB4  NaN  NaN  NaN  NaN   ABCB4  ...   
2                DRN  A1MD34  NaN     NaN  NaN  NaN  NaN  NaN     NaN  ...   
3                 ON     NaN  NaN     NaN  NaN  NaN  NaN  NaN     NaN  ...   
4             UNT N2     NaN  NaN  TIET11  NaN  NaN  NaN  NaN  TIET11  ...   

                                                                 
                                                                 
  IGNM IMAT IMOB INDX    ISEE    ITAG IVBX MLCX    SMLL    UTIL  
0  NaN  NaN  NaN  NaN     NaN     NaN  NaN  NaN     NaN     NaN  
1  NaN  NaN  NaN  NaN     NaN   ABCB4  NaN  NaN   ABCB4     NaN  
2  NaN  NaN  NaN  NaN     NaN     NaN  NaN  NaN     NaN     NaN  
3  NaN  NaN  NaN  NaN     NaN   ADHM3  NaN  NaN     NaN     NaN  
4  NaN  NaN  NaN  NaN  TIET11  TIET11  NaN  NaN  TIET11  TIET11  

[5 rows x 25 columns]

In [5]:
# Fix MultiIndex on columns
stock_by_indx.columns = [col[2] for col in stock_by_indx.columns]

In [6]:
stock_by_indx.head()

,Unnamed: 0_level_2,Unnamed: 1_level_2,BDRX,IBOV,IBRA,IBXL,IBXX,ICO2,ICON,IDIV,...,IGNM,IMAT,IMOB,INDX,ISEE,ITAG,IVBX,MLCX,SMLL,UTIL
0,3M,DRN,MMMC34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABC BRASIL,PN N2,NaN,NaN,ABCB4,NaN,NaN,NaN,NaN,ABCB4,...,NaN,NaN,NaN,NaN,NaN,ABCB4,NaN,NaN,ABCB4,NaN
2,ADVANCED MIC,DRN,A1MD34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ADVANCED-DH,ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ADHM3,NaN,NaN,NaN,NaN
4,AES TIETE E,UNT N2,NaN,NaN,TIET11,NaN,NaN,NaN,NaN,TIET11,...,NaN,NaN,NaN,NaN,TIET11,TIET11,NaN,NaN,TIET11,TIET11


In [77]:
stock_by_indx.drop(stock_by_indx.columns[:2], axis=1, inplace=True)

## Selecionando os índices com o maior número de ativos

In [64]:
qtd_stocks_indx = pd.DataFrame(stock_by_indx.iloc[-1])
qtd_stocks_indx.columns = ['qtd']
qtd_stocks_indx.qtd = qtd_stocks_indx.qtd.astype('int64')

In [75]:
selected_indx = qtd_stocks_indx.sort_values(by='qtd', ascending=False).head(5).index.tolist()

['IGCX', 'ITAG', 'IBRA', 'IGCT', 'IGNM']

In [18]:
df_selected_indx = stock_by_indx[selected_indx].copy()

In [41]:
list_stocks = []
for col in selected_indx:
    list_stocks.append(stock_by_indx[col].values.tolist())

In [44]:
list_stocks = list_stocks[0]

In [82]:
list_unique_stocks = list(set(list_stocks))

Removendo valores que não fazem parte do conjunto de ativos

In [88]:
list_unique_stocks.remove(np.nan)
list_unique_stocks.remove('206')

## Baixando dados das cotações

In [97]:
from pandas_datareader import DataReader
from tqdm import tqdm

In [99]:
start_date = '2017-09-01'
final_date = '2020-09-01'
stocks_quotes = []

for stock in tqdm(list_unique_stocks):
    df = DataReader('%s.SA' % stock, 'yahoo', start_date, final_date)
    stocks_quotes.append(df)


 53%|█████▎    | 110/206 [06:34<05:43,  3.58s/it]


KeyError: 'Date'

### Questão 2: 

### Questão 3: 

### Questão 4: 

### Questão 5: 

### Questão 6: 